In [1]:
import numpy as np
import math
import os
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
import random
from joblib import Parallel, delayed

In [2]:
##parameter setting
trans_mat=np.array([[0.6,0.4],[0.8,0.2]])
trans_mat_new=np.array([[0.5,0.5],[0.7,0.3]])


In [3]:
#################################
###### move one step forward ####
#################################


def step(last_obs):
    # last_obs: last observation of state

    if last_obs == 1:
        a = np.random.binomial(1, p=trans_mat[0,1])
        r = np.random.normal(2, 1)  # reward at the same stage
        s_next = a + 1  # next state
    elif last_obs == 2:
        a = np.random.binomial(1, p=trans_mat[1,1])
        r = np.random.normal(-1, 1)
        s_next = a + 1

    return (a, r, s_next)

def step_new(last_obs):
    # last_obs: last observation of state
    if last_obs == 1:
        a = np.random.binomial(1, p=trans_mat_new[0,1])
        r = np.random.normal(2, 1)  # reward at the same stage
        s_next = a + 1  # next state
    elif last_obs == 2:
        a = np.random.binomial(1, p=trans_mat_new[1,1])
        r = np.random.normal(-1, 1)
        s_next = a + 1

    return (a, r, s_next)

In [4]:
#################################
#### generate one trajectory ####
#################################


def gen_traj(T, gam, seed=None, s_init=None):
    # seed: random seed
    # s_init: initial state
    # gam: discount
    # T: iterative number

    # initialize the state
    if seed is None and s_init is None:
        s = np.random.binomial(1, p=0.5) + 1
    elif seed is not None:
        np.random.seed(seed)
        s = np.random.binomial(1, p=0.5) + 1
    if s_init is not None:
        s = s_init

    s_traj = [s]
    a_traj = []
    r_traj = []

    ret = 0
    for i in range(T):
        a, r, s_next = step(s)
        s_traj.append(s_next)
        a_traj.append(a)
        r_traj.append(r)
        s = s_next  # update current S as S_next
        ret += r * gam**i

    ## output state, reward trajectory. return
    return [s_traj, a_traj, r_traj, ret]


def gen_traj_new(T, gam, seed=None, s_init=None):
    # seed: random seed
    # s_init: initial state
    # gam: discount
    # T: iterative number

    # initialize the state
    if seed is None and s_init is None:
        s = np.random.binomial(1, p=0.5) + 1
    elif seed is not None:
        np.random.seed(seed)
        s = np.random.binomial(1, p=0.5) + 1
    if s_init is not None:
        s = s_init

    s_traj = [s]
    a_traj = []
    r_traj = []

    ret = 0
    for i in range(T):
        a, r, s_next = step_new(s)
        s_traj.append(s_next)
        a_traj.append(a)
        r_traj.append(r)
        s = s_next  # update current S as S_next
        ret += r * gam**i

    ## output state, reward trajectory. return
    return [s_traj, a_traj, r_traj, ret]

In [5]:
#######################
#### generate data ####
#######################


def data_gen(N, T_obs, T, gam, seed=None, s_init=None):
    # N: number of trajectories
    # T_obs: observed stage numbers

    s_data = np.zeros((N, T_obs), dtype=int)
    a_data = np.zeros((N, T_obs), dtype=int)
    r_data = np.zeros((N, T_obs))
    ret_data = []

    for i in range(N):
        if seed is not None:
            seed += 1
        tmp = gen_traj(T, gam, seed, s_init)
        s_data[i] = tmp[0][0:T_obs]  # store the i-th state trajectory
        a_data[i] = tmp[1][0:T_obs]
        r_data[i] = tmp[2][0:T_obs]  # store the i-th reward trajectory
        ret_data.append(tmp[3])

    ## output observed state, reward trajectory and true return
    return [s_data, a_data ,r_data, ret_data]

def data_gen_new(N, T_obs, T, gam, seed=None, s_init=None):
    # N: number of trajectories
    # T_obs: observed stage numbers

    s_data = np.zeros((N, T_obs), dtype=int)
    a_data = np.zeros((N, T_obs), dtype=int)
    r_data = np.zeros((N, T_obs))
    ret_data = []

    for i in range(N):
        if seed is not None:
            seed += 1
        tmp = gen_traj_new(T, gam, seed, s_init)
        s_data[i] = tmp[0][0:T_obs]  # store the i-th state trajectory
        a_data[i] = tmp[1][0:T_obs]
        r_data[i] = tmp[2][0:T_obs]  # store the i-th reward trajectory
        ret_data.append(tmp[3])

    ## output observed state, reward trajectory and true return
    return [s_data, a_data ,r_data, ret_data]

In [24]:
#######################
## simulation setup ###
#######################

n_tr = 100
gam = 0.8
T_obs = 30
seed = 2025

data_train = data_gen(N=n_tr,
                      T_obs=T_obs,
                      T=100,
                      gam=gam,
                      seed=seed,
                      s_init=None)



In [6]:
####################################
### Quantile temperal difference ###
####################################


def QTD(state_traj,
        action_traj,
        reward_traj,
        state_card,
        action_card,
        quantile_num,
        gam,
        rate=None,
        init_val=None):
    # obs_traj: training data
    # state_card: cardinality of state space
    ## quantile_num: number of target conditional quantiles for each state
    # rate: learning rate
    # init_val: initial value

    if init_val == None:
        ret_con_quantile = np.zeros((state_card,action_card, quantile_num))
    elif init_value is not None:
        ret_con_quantile = init_value
    if rate == None:
        rate = 0.1

    #ret_con_quantile = np.zeros((state_card, quantile_num))

    n = np.shape(reward_traj)[0]  ## number of trajectories
    batch_num = np.shape(state_traj)[1] - 1  ## number of (x,r,x') tuples
    tau = [(2 * i + 1) / (2 * quantile_num) for i in range(quantile_num)]
    
    for k in range(n):
        for l in range(batch_num):
            for i in range(quantile_num):    
                s_current = state_traj[k, l]
                a_current = action_traj[k, l]
                r_current = reward_traj[k, l]
                s_next = state_traj[k, l + 1]
                a_next = action_traj[k, l + 1]
                #j = np.random.randint(0, quantile_num - 1)
                #ret_con_quantile[s_current - 1, i] += rate * tau[i] - rate * (
                #    r_current + gam * ret_con_quantile[s_next - 1, j] -
                #    ret_con_quantile[s_current - 1, i] < 0)
                ret_con_quantile[s_current - 1, a_current, i] +=  rate * np.mean(
                    [tau[i] - 1 * (r_current + gam * 
                                      ret_con_quantile[s_next - 1,a_next, j] - 
                                      ret_con_quantile[s_current - 1, a_current, i] < 0) 
                     for j in range(quantile_num)] 
                )

    ## output conditional quantiles
    return (ret_con_quantile)

def QTD_new(state_traj,
        action_traj,
        reward_traj,
        state_card,
        action_card,
        quantile_num,
        gam,
        seed,
        rate=None,
        init_val=None):
    # obs_traj: training data
    # state_card: cardinality of state space
    ## quantile_num: number of target conditional quantiles for each state
    # rate: learning rate
    # init_val: initial value

    if init_val == None:
        ret_con_quantile = np.zeros((state_card,action_card, quantile_num))
    elif init_value is not None:
        ret_con_quantile = init_value
    if rate == None:
        rate = 0.1

    #ret_con_quantile = np.zeros((state_card, quantile_num))
    
    n = np.shape(reward_traj)[0]  ## number of trajectories
    batch_num = np.shape(state_traj)[1] - 1  ## number of (x,a,r,x') tuples
    tau = [(2 * i + 1) / (2 * quantile_num) for i in range(quantile_num)]
    
    for k in range(n):
        for l in range(batch_num):
            seed+=1
            np.random.seed(seed)
            s_current = state_traj[k, l]
            a_current = action_traj[k, l]
            r_current = reward_traj[k, l]
            s_next = state_traj[k, l + 1]
            a_next = step_new(s_next)[0] #generate according to pi_a
            for i in range(quantile_num):    
               
                #j = np.random.randint(0, quantile_num - 1)
                #ret_con_quantile[s_current - 1, i] += rate * tau[i] - rate * (
                #    r_current + gam * ret_con_quantile[s_next - 1, j] -
                #    ret_con_quantile[s_current - 1, i] < 0)
                ret_con_quantile[s_current - 1, a_current, i] +=  rate * np.mean(
                    [tau[i] - 1 * (r_current + gam * 
                                      ret_con_quantile[s_next - 1,a_next, j] - 
                                      ret_con_quantile[s_current - 1, a_current, i] < 0)
                     for j in range(quantile_num)] 
                )

    ## output conditional quantiles
    return (ret_con_quantile)

In [7]:
## calculate V estimator based on QTD output
def q_hat_f(G_quan):
    return (np.mean(G_quan, axis=2))




In [8]:
def weighted_percentile(data, weights, perc):

    data = np.array(data)
    weights = np.array(weights)
    idx = np.argsort(data)
    data = data[idx] # sort data
    weights = weights[idx] # sort weights
    cdf = np.cumsum(weights) / np.sum(weights)
    count = np.sum([ cdf[i] <= perc for i in range(np.shape(cdf)[0]) ])
    #if output=infty return the maximum of V
    if data[count]==float('inf'):
        count-=1
    return(data[count])
    

In [9]:
def CQR_quantile(alp,G_quan):
    s=np.shape(G_quan)[0]
    quan_num=np.shape(G_quan)[2]
    res=np.zeros((s,np.shape(G_quan)[1]))
    for i in range(s):
        data_aug=np.hstack([G_quan[i,0],G_quan[i,1]])
        weight_aug=np.hstack([np.ones(quan_num)*trans_mat_new[i,0],np.ones(quan_num)*trans_mat_new[i,1]])
        res[i,0]=weighted_percentile(data_aug, weight_aug, alp/2)
        res[i,1]=weighted_percentile(data_aug, weight_aug, 1-alp/2)
    return (res)

In [10]:
#计算pi_a(a|x)
p1=(trans_mat_new[0,1]/trans_mat_new[0,0])*(trans_mat[0,0]/trans_mat[0,1])
a1=p1/(1+p1)
p2=(trans_mat_new[1,1]/trans_mat_new[1,0])*(trans_mat[1,0]/trans_mat[1,1])
a2=p2/(1+p2)
def step_a(last_obs):
    if last_obs == 1:
        a = np.random.binomial(1, p=a1)
    elif last_obs == 2:
        a = np.random.binomial(1, p=a2)

    return (a)
    
vec_step=np.vectorize(step_a)
mat=np.array([[1-a1,a1],[1-a2,a2]])
weight_mat=trans_mat_new/trans_mat
def weight_calculation_clip(s_vec, a_vec,clip):
    weight=1
    step=np.shape(s_vec)[0]-1
    for i in range(step):
        weight*=weight_mat[s_vec[i]-1,a_vec[i]]
    weight=max(min(clip[1],weight),clip[0])
    return(weight)

In [11]:
## replay buffer+propensity score ratio
def replay_buffer(s_traj, a_traj, r_traj,step_forward,ratio,clip):
    n = np.shape(s_traj)[0]
    p = np.shape(s_traj)[1] - step_forward 
    Mem_state = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_action_a = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_action = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_reward = np.zeros((n*p,step_forward+1))
    idx_weight=[]
    for i in range(n):
        for j in range(p):
            Mem_state[(i*p+j),:] = s_traj[i,j:(j+step_forward+1)]
            Mem_action_a[(i*p+j),:] = vec_step(s_traj[i,j:(j+step_forward+1)])
            Mem_action[(i*p+j),:] = a_traj[i,j:(j+step_forward+1)]
            Mem_reward[(i*p+j),:] = r_traj[i,j:(j+step_forward+1)]
            idx_weight.append(
            weight_calculation_clip(Mem_state[(i*p+j), :], Mem_action[(i*p+j), :],clip) * ratio[Mem_state[(i*p+j), 0]-1]
            )

    total = np.array(idx_weight).sum()
    idx_weight_final=np.array(idx_weight)/total
    return([Mem_state,Mem_action,Mem_reward,idx_weight_final])

##ratio of frequency of s_0/s_rb
def rb(s_traj, a_traj, r_traj,step_forward):
    n = np.shape(s_traj)[0]
    p = np.shape(s_traj)[1] - step_forward 
    Mem_state = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_action_a = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_action = np.zeros((n*p,step_forward+1),dtype=int)
    Mem_reward = np.zeros((n*p,step_forward+1))
    acc_1=0
    acc_2=0
    for i in range(n):
        for j in range(p):
            Mem_state[(i*p+j),:] = s_traj[i,j:(j+step_forward+1)]
            Mem_action_a[(i*p+j),:] = vec_step(s_traj[i,j:(j+step_forward+1)])
            Mem_action[(i*p+j),:] = a_traj[i,j:(j+step_forward+1)]
            Mem_reward[(i*p+j),:] = r_traj[i,j:(j+step_forward+1)]
            if (vec_step(s_traj[i,j:(j+step_forward)])==a_traj[i,j:(j+step_forward)]).all() and Mem_state[(i*p+j),0]==1 :
                acc_1+=1
            elif (vec_step(s_traj[i,j:(j+step_forward)])==a_traj[i,j:(j+step_forward)]).all() and Mem_state[(i*p+j),0]==2 :
                acc_2+=1
    ratio_1=(2*n*p-np.sum(Mem_state[:,0]))/acc_1
    ratio_2=(np.sum(Mem_state[:,0])-n*p)/acc_2
    return([ratio_1,ratio_2])



In [12]:
def scoring(s_traj, r_traj,step_forward,gam,G_quan,v_hat):
    # s_traj: state trajectory
    # r_traj: reward trajectory
    # step_forward: number of steps used in approximating return
    # gam: discount
    if np.shape(s_traj)[1]!=step_forward+1:
        print("length dismatch")
    if np.shape(s_traj)[0]!=np.shape(r_traj)[0]:
        print("height dismatch")
    
    quan_num = np.shape(G_quan)[2]
    n = np.shape(s_traj)[0]
    u = np.random.randint(0, quan_num - 1, size=n)
    sc = list(
        map(
            abs,
            np.sum([gam**i * r_traj[:, i] for i in range(step_forward)],
                   axis=0) +
            [
                gam**step_forward * G_quan[s_traj[i, step_forward] - 1, step_new(s_traj[i, step_forward])[0],u[i]] -
                v_hat[s_traj[i, 0] - 1] for i in range(n)
            ]))

    return (sc)

In [23]:
def scoring_CQR(s_traj, r_traj,step_forward,gam,G_quan,CQR_quan):
    if np.shape(s_traj)[1]!=step_forward+1:
        print("length dismatch")
    if np.shape(s_traj)[0]!=np.shape(r_traj)[0]:
        print("height dismatch")
    
    quan_num = np.shape(G_quan)[2]
    n = np.shape(s_traj)[0]
    u = np.random.randint(0, quan_num - 1, size=n)
    sc = list(
        map(
            max,
            zip(
            -np.sum([gam**i * r_traj[:, i] for i in range(step_forward)],
                   axis=0) +
            [
                 
                CQR_quan[s_traj[i, 0] - 1,0]-gam**step_forward * G_quan[s_traj[i, step_forward] - 1, step_new(s_traj[i, step_forward])[0],u[i]] 
                for i in range(n)
            ],
                
            np.sum([gam**i * r_traj[:, i] for i in range(step_forward)],
                   axis=0) +
            [
                gam**step_forward * G_quan[s_traj[i, step_forward] - 1, step_new(s_traj[i, step_forward])[0],u[i]] -
                CQR_quan[s_traj[i, 0] - 1,1] for i in range(n)
            ]   
                
                
                
            )))

    return (sc)

## replicate experiment

In [24]:
import itertools

In [31]:
def new_rb_res(data_train, data_test, gam, alp, step_forward, QTD_para,B,tau,seed,sample_size,clip):
    n_tr, n_te = np.shape(data_train[0])[0], np.shape(data_test[0])[0]
    s_init_te = data_test[0]
    a_init_te = data_test[1]
    ret_te = data_test[3]
    
    ## split training data
    idx_perm = np.random.permutation(list(range(0, n_tr)))
    idx_tr, idx_cal = [idx_perm[0:int(n_tr / 2)], idx_perm[int(n_tr / 2):n_tr]]
    s_train_fold = data_train[0][idx_tr,:]
    a_train_fold = data_train[1][idx_tr,:]
    r_train_fold = data_train[2][idx_tr,:]
    
    ## train return distribution using QTD
    G_quan = QTD_new(state_traj=s_train_fold,
                 action_traj=a_train_fold,
                 reward_traj=r_train_fold,
                 state_card=2,
                 action_card=2,
                 quantile_num=QTD_para[0],
                 gam=gam,
                 seed=seed,
                 rate=QTD_para[1],
                 init_val=None)
    Q_hat = q_hat_f(G_quan)
    v_hat = np.zeros(2)
    v_hat[0]=trans_mat_new[0,0]*Q_hat[0,0]+trans_mat_new[0,1]*Q_hat[0,1]
    v_hat[1]=trans_mat_new[1,0]*Q_hat[1,0]+trans_mat_new[1,1]*Q_hat[1,1]
    CQR_quan = CQR_quantile(alp,G_quan)


    ## calculate nonconformity scores based on test set
    sc_te = [abs(ret_te[i] - v_hat[s_init_te[i, 0] - 1]) for i in range(n_te)]
    sc_te_cqr = [max((ret_te[i] - CQR_quan[s_init_te[i, 0] - 1, 1]),CQR_quan[s_init_te[i, 0] - 1, 0] - ret_te[i]  ) for i in range(n_te)]
    
    ## replay buffer
    l = np.shape(step_forward)[0]
    if isinstance(tau, int) == False:
        m = np.shape(tau)[0]
    elif isinstance(tau, int) == True:
        m = 1
        
    PI_cov_e = np.zeros((m,l))
    PI_len_e = np.zeros((m,l))
    PI_cov_c = np.zeros((m,l))
    PI_len_c = np.zeros((m,l))
            
    p1_s0_estimate = np.mean(s_train_fold[:,0] - 1)

            
        
    for k in range(l):
       
        ratio=rb(s_traj=data_train[0][idx_tr, :],
                    a_traj=data_train[1][idx_tr, :],
               r_traj=data_train[2][idx_tr, :],
                step_forward=step_forward[k])
        ## density ratio
        Mem_1 = replay_buffer(s_traj=s_train_fold,
                              a_traj=a_train_fold,
                              r_traj=r_train_fold,
                              step_forward=1,
                             ratio=np.ones(2),
                             clip=np.ones(2))
        p1_rb = np.mean(Mem_1[0][:,0] - 1)
        dr_s_e = [((1-p1_s0_estimate)/(1-p1_rb))*ratio[0],(p1_s0_estimate/p1_rb)*ratio[1]]
        
        quan_B_e = np.zeros((m,n_te,B))
        quan_B_c = np.zeros((m,n_te,B))
        

        for i in range(B):
            
        #n_cal = np.random.choice(a=[j for j in range(np.shape(Mem[0])[0])], p=p,size=200)
            Mem = replay_buffer(s_traj=data_train[0][idx_cal, :],
                    a_traj=data_train[1][idx_cal, :],
                 r_traj=data_train[2][idx_cal, :],
                step_forward=step_forward[k],
                               ratio=ratio,
                               clip=clip)
            weight_is=Mem[-1]
            n_cal = np.random.choice(range(np.shape(weight_is)[0]),size=sample_size, p=weight_is)
            ## calculate nonconformity scores based on calibration set
            sc_rb = scoring(s_traj=Mem[0][n_cal,],
                            r_traj=Mem[2][n_cal,],
                            step_forward=step_forward[k],
                            gam=gam,
                            G_quan=G_quan,
                            v_hat=v_hat)
            sc_cqr = scoring_CQR(s_traj=Mem[0][n_cal,],
                            r_traj=Mem[2][n_cal,],
                            step_forward=step_forward[k],
                            gam=gam,
                            G_quan=G_quan,
                            CQR_quan=CQR_quan
                                )
            sc_rb.append(float('inf')) 
            sc_cqr.append(float('inf')) 
            for j in range(n_te): 
                for z in range(m):
                    quan_B_e[z][j,i] = weighted_percentile(data=sc_rb,weights=np.ones(sample_size+1),
                                                      perc=1-alp)
                    quan_B_c[z][j,i] = weighted_percentile(data=sc_cqr,weights=np.ones(sample_size+1),
                                                      perc=1-alp)
                    
        critical_value_rb_e = np.zeros((m,n_te))
        critical_value_rb_c = np.zeros((m,n_te))
        
 
        for z in range(m):
            critical_value_rb_e[z,:] = [ np.percentile(a=quan_B_e[z][k,:],
                                                           q=tau[z]*100) for k in range(n_te) ]

            critical_value_rb_c[z,:] = [ np.percentile(a=quan_B_c[z][k,:],
                                                           q=tau[z]*100) for k in range(n_te) ]

            
            PI_cov_e[z,k] = np.mean([sc_te[k] <= critical_value_rb_e[z,k] 
                                         for k in range(n_te)])
            PI_len_e[z,k] = 2 * np.mean(critical_value_rb_e[z,:])
            PI_cov_c[z,k] = np.mean([sc_te_cqr[k] <= critical_value_rb_c[z,k] 
                                         for k in range(n_te)])
            PI_len_c[z,k] = 2 * np.mean(critical_value_rb_c[z,:]) + np.mean([CQR_quan[s_init_te[i, 0] - 1, 1] - CQR_quan[s_init_te[i, 0] - 1, 0]  for k in range(n_te) ])
            
            
    return(np.array([[PI_cov_e,PI_len_e], [PI_cov_c,PI_len_c]]))


In [35]:
def quantile_region_res(data_train, data_test, gam, alp, QTD_para,seed):

    n_tr, n_te = np.shape(data_train[0])[0], np.shape(data_test[0])[0]
    s_init_te = data_test[0]
    a_init_te = data_test[1] 
    ret_te = data_test[3]
    quant_num = QTD_para[0]
    ## train QTD using full training data
    G_quan_full = QTD_new(state_traj=data_train[0],
                      action_traj=data_train[1],
                      reward_traj=data_train[2],
                      state_card=2,
                      action_card=2,  
                      quantile_num=quant_num,
                      gam=gam,
                      seed=seed,
                      rate=QTD_para[1],
                      init_val=None)
    quant_interval_lower=np.zeros(2)
    quant_interval_upper=np.zeros(2)

    ## lower and upper quanitles for each states 
    for i in range(2):
        data_aug=np.hstack((G_quan_full[i,0,:], G_quan_full[i,1,:])) 
        weight_aug=np.hstack((np.ones(quant_num)*trans_mat_new[i,0]/quant_num,np.ones(quant_num)*trans_mat_new[i,1]/quant_num))
        quant_interval_lower[i]=weighted_percentile(data_aug,weight_aug,alp/2)
        quant_interval_upper[i]=weighted_percentile(data_aug,weight_aug,1-alp/2)
                       

    ## calculate coverage
    t1 = [
    ret_te[i] >= quant_interval_lower[s_init_te[i, 0] - 1] for i in range(n_te)
    ]
    t2 = [
    ret_te[i] <= quant_interval_upper[s_init_te[i, 0] - 1] for i in range(n_te)
    ]
    quan_PI_cov = np.mean([all([t1[i], t2[i]]) for i in range(n_te)])
    quan_PI_len = np.mean([
        quant_interval_upper[s_init_te[i, 0] - 1] -
        quant_interval_lower[s_init_te[i, 0] - 1] for i in range(n_te)
        ])

    return ([quan_PI_cov, quan_PI_len])

In [36]:
#Parallel Calculation
def run_single_experiment(i, n_tr, gam, T_obs, seed, n_te, T, QTD_para, B, alp, tau, step_forward,sample_size,clip):

    data_train = data_gen(N=n_tr,
                          T_obs=T_obs,
                          T=T,
                          gam=gam,
                          seed=seed + i,
                          s_init=None)


    data_test = data_gen_new(N=n_te,
                             T_obs=1,
                             T=T,
                             gam=gam,
                             seed=seed + i + 10000,
                             s_init=None)

    result = new_rb_res(data_train=data_train,
                        data_test=data_test,
                        gam=gam,
                        alp=alp,
                        step_forward=step_forward,
                        QTD_para=QTD_para,
                        B=B,
                        tau=tau,
                        seed=seed + i,
                        sample_size=sample_size,
                       clip=clip)
   
    return result  # return [PI_cov_e, PI_len_e]




In [38]:
# Parameter setting
rep = 50
n_tr = 400
gam = 0.8
T_obs = 30
seed = 2025
n_te = 310
T = 70
QTD_para = [20, 0.1]
B = 100
alp = 0.1
tau = [0.5,0.6,0.7,0.8,0.9]
clip=np.array([0.2,5.0])
step_forward = [1, 2, 3,4,5]
sample_size=400




results = Parallel(n_jobs=26, verbose=1)(
    delayed(run_single_experiment)(
        i, n_tr, gam, T_obs, seed, n_te, T, QTD_para, B, alp, tau, step_forward, sample_size,clip
    ) for i in range(rep)
)



[Parallel(n_jobs=26)]: Using backend LokyBackend with 26 concurrent workers.
[Parallel(n_jobs=26)]: Done  50 out of  50 | elapsed: 54.8min finished


In [41]:
results[0][1][0][0]

array([0.89677419, 0.90322581, 0.90322581, 0.90322581, 0.90322581])

In [44]:

rb_new_cov_tau05_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau06_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau07_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau08_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau09_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau05_c3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau06_c3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau07_c3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau08_c3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_cov_tau09_c3 = np.zeros((rep, np.shape(step_forward)[0]))

rb_new_len_tau05_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau06_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau07_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau08_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau09_e3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau05_c3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau06_c3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau07_c3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau08_c3 = np.zeros((rep, np.shape(step_forward)[0]))
rb_new_len_tau09_c3 = np.zeros((rep, np.shape(step_forward)[0]))


#Restore data
for i in range(np.shape(results)[0]):
    rb_new_cov_tau05_e3[i, :] = results [i][0][0][0]
    rb_new_cov_tau06_e3[i, :] = results [i][0][0][1]
    rb_new_cov_tau07_e3[i, :] = results [i][0][0][2]
    rb_new_cov_tau08_e3[i, :] = results [i][0][0][3]
    rb_new_cov_tau09_e3[i, :] = results [i][0][0][4]
    rb_new_cov_tau05_c3[i, :] = results [i][1][0][0]
    rb_new_cov_tau06_c3[i, :] = results [i][1][0][1]
    rb_new_cov_tau07_c3[i, :] = results [i][1][0][2]
    rb_new_cov_tau08_c3[i, :] = results [i][1][0][3]
    rb_new_cov_tau09_c3[i, :] = results [i][1][0][4]



    rb_new_len_tau05_e3[i, :] = results [i][0][1][0]
    rb_new_len_tau06_e3[i, :] = results [i][0][1][1]
    rb_new_len_tau07_e3[i, :] = results [i][0][1][2]
    rb_new_len_tau08_e3[i, :] = results [i][0][1][3]
    rb_new_len_tau09_e3[i, :] = results [i][0][1][4]
    rb_new_len_tau05_c3[i, :] = results [i][1][1][0]
    rb_new_len_tau06_c3[i, :] = results [i][1][1][1]
    rb_new_len_tau07_c3[i, :] = results [i][1][1][2]
    rb_new_len_tau08_c3[i, :] = results [i][1][1][3]
    rb_new_len_tau09_c3[i, :] = results [i][1][1][4]


PI_cov_all = [res[0] for res in results]
PI_len_all = [res[1] for res in results]

In [19]:
import pandas as pd
print("new(tau=0.5): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau05_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau05_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])

print("new(tau=0.6): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau06_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau06_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])

print("new(tau=0.7): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau07_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau07_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])

print("new(tau=0.8): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau08_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau08_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])


print("new(tau=0.9): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau09_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau09_e3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])



new(tau=0.5): 
coverage probability: 
[0.8854193548387098, 0.8949677419354838, 0.8990322580645161, 0.8998709677419354, 0.901032258064516]
average length: 
[7.345789251332047, 7.527736699385553, 7.591523913105704, 7.617437787121841, 7.633841954903639]
new(tau=0.6): 
coverage probability: 
[0.8893548387096775, 0.8998709677419356, 0.9047096774193548, 0.9059354838709678, 0.9065806451612903]
average length: 
[7.419672341433534, 7.60546178408489, 7.672430586289693, 7.695741294964429, 7.718315209717754]
new(tau=0.7): 
coverage probability: 
[0.8938709677419354, 0.9056129032258065, 0.9098709677419354, 0.9118064516129032, 0.9119999999999999]
average length: 
[7.504068875714554, 7.6943841995042375, 7.761487934075203, 7.789678237119019, 7.805704300440607]
new(tau=0.8): 
coverage probability: 
[0.9007096774193549, 0.9125161290322581, 0.9163870967741936, 0.9184516129032257, 0.9198709677419356]
average length: 
[7.606103286530054, 7.792725281603938, 7.859996165452284, 7.89360678105463, 7.91070326863

In [45]:
print("new(tau=0.5): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau05_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau05_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])

print("new(tau=0.6): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau06_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau06_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])

print("new(tau=0.7): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau07_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau07_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])

print("new(tau=0.8): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau08_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau08_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])


print("new(tau=0.9): ")
print("coverage probability: ")
print([np.mean(rb_new_cov_tau09_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])
print("average length: ")
print([np.mean(rb_new_len_tau09_c3[:,i]) for i in range(np.shape(rb_new_cov_tau05_e3)[1])])

new(tau=0.5): 
coverage probability: 
[0.8923870967741935, 0.8983870967741936, 0.9016129032258065, 0.9018064516129032, 0.9025806451612902]
average length: 
[7.3557549944269205, 7.504955885529669, 7.5676093314613935, 7.565621839564374, 7.582240681034324]
new(tau=0.6): 
coverage probability: 
[0.8953548387096774, 0.902, 0.9058064516129031, 0.905548387096774, 0.9065806451612903]
average length: 
[7.433492506936812, 7.589868212387205, 7.654981831947721, 7.646651641679553, 7.667079998608845]
new(tau=0.7): 
coverage probability: 
[0.8977419354838709, 0.9067096774193548, 0.9104516129032258, 0.9096774193548387, 0.9113548387096775]
average length: 
[7.512933505270488, 7.677102466595763, 7.74353349694107, 7.73352535064873, 7.750900159162187]
new(tau=0.8): 
coverage probability: 
[0.9029032258064515, 0.9123225806451611, 0.9172258064516129, 0.9173548387096774, 0.9175483870967741]
average length: 
[7.6178597171025215, 7.786373561940102, 7.856812116541559, 7.84650303183024, 7.856703493884708]
new(ta

In [48]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau05_e3[:,0],
    'new-rb-2': rb_new_cov_tau05_e3[:,1],
    'new-rb-3': rb_new_cov_tau05_e3[:,2],
    'new-rb-4': rb_new_cov_tau05_e3[:,3],
    'new-rb-5': rb_new_cov_tau05_e3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_05.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau05_e3[:,0],
     'new-rb-2': rb_new_len_tau05_e3[:,1],
     'new-rb-3': rb_new_len_tau05_e3[:,2],
     'new-rb-4': rb_new_len_tau05_e3[:,3],
     'new-rb-5': rb_new_len_tau05_e3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_05.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [49]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau05_c3[:,0],
    'new-rb-2': rb_new_cov_tau05_c3[:,1],
    'new-rb-3': rb_new_cov_tau05_c3[:,2],
    'new-rb-4': rb_new_cov_tau05_c3[:,3],
    'new-rb-5': rb_new_cov_tau05_c3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_cqr_05.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau05_c3[:,0],
     'new-rb-2': rb_new_len_tau05_c3[:,1],
     'new-rb-3': rb_new_len_tau05_c3[:,2],
     'new-rb-4': rb_new_len_tau05_c3[:,3],
     'new-rb-5': rb_new_len_tau05_c3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_cqr_05.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [50]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau06_e3[:,0],
    'new-rb-2': rb_new_cov_tau06_e3[:,1],
    'new-rb-3': rb_new_cov_tau06_e3[:,2],
    'new-rb-4': rb_new_cov_tau06_e3[:,3],
    'new-rb-5': rb_new_cov_tau06_e3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_06.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau06_e3[:,0],
     'new-rb-2': rb_new_len_tau06_e3[:,1],
     'new-rb-3': rb_new_len_tau06_e3[:,2],
     'new-rb-4': rb_new_len_tau06_e3[:,3],
     'new-rb-5': rb_new_len_tau06_e3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_06.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [51]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau06_c3[:,0],
    'new-rb-2': rb_new_cov_tau06_c3[:,1],
    'new-rb-3': rb_new_cov_tau06_c3[:,2],
    'new-rb-4': rb_new_cov_tau06_c3[:,3],
    'new-rb-5': rb_new_cov_tau06_c3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_cqr_06.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau06_c3[:,0],
     'new-rb-2': rb_new_len_tau06_c3[:,1],
     'new-rb-3': rb_new_len_tau06_c3[:,2],
     'new-rb-4': rb_new_len_tau06_c3[:,3],
     'new-rb-5': rb_new_len_tau06_c3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_cqr_06.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [52]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau07_e3[:,0],
    'new-rb-2': rb_new_cov_tau07_e3[:,1],
    'new-rb-3': rb_new_cov_tau07_e3[:,2],
    'new-rb-4': rb_new_cov_tau07_e3[:,3],
    'new-rb-5': rb_new_cov_tau07_e3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_07.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau07_e3[:,0],
     'new-rb-2': rb_new_len_tau07_e3[:,1],
     'new-rb-3': rb_new_len_tau07_e3[:,2],
     'new-rb-4': rb_new_len_tau07_e3[:,3],
     'new-rb-5': rb_new_len_tau07_e3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_07.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [53]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau07_c3[:,0],
    'new-rb-2': rb_new_cov_tau07_c3[:,1],
    'new-rb-3': rb_new_cov_tau07_c3[:,2],
    'new-rb-4': rb_new_cov_tau07_c3[:,3],
    'new-rb-5': rb_new_cov_tau07_c3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_cqr_07.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau07_c3[:,0],
     'new-rb-2': rb_new_len_tau07_c3[:,1],
     'new-rb-3': rb_new_len_tau07_c3[:,2],
     'new-rb-4': rb_new_len_tau07_c3[:,3],
     'new-rb-5': rb_new_len_tau07_c3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_cqr_07.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [54]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau08_e3[:,0],
    'new-rb-2': rb_new_cov_tau08_e3[:,1],
    'new-rb-3': rb_new_cov_tau08_e3[:,2],
    'new-rb-4': rb_new_cov_tau08_e3[:,3],
    'new-rb-5': rb_new_cov_tau08_e3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_08.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau08_e3[:,0],
     'new-rb-2': rb_new_len_tau08_e3[:,1],
     'new-rb-3': rb_new_len_tau08_e3[:,2],
     'new-rb-4': rb_new_len_tau08_e3[:,3],
     'new-rb-5': rb_new_len_tau08_e3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_08.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [55]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau08_c3[:,0],
    'new-rb-2': rb_new_cov_tau08_c3[:,1],
    'new-rb-3': rb_new_cov_tau08_c3[:,2],
    'new-rb-4': rb_new_cov_tau08_c3[:,3],
    'new-rb-5': rb_new_cov_tau08_c3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_cqr_08.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau08_c3[:,0],
     'new-rb-2': rb_new_len_tau08_c3[:,1],
     'new-rb-3': rb_new_len_tau08_c3[:,2],
     'new-rb-4': rb_new_len_tau08_c3[:,3],
     'new-rb-5': rb_new_len_tau08_c3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_cqr_08.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [56]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau09_e3[:,0],
    'new-rb-2': rb_new_cov_tau09_e3[:,1],
    'new-rb-3': rb_new_cov_tau09_e3[:,2],
    'new-rb-4': rb_new_cov_tau09_e3[:,3],
    'new-rb-5': rb_new_cov_tau09_e3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_09.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau09_e3[:,0],
     'new-rb-2': rb_new_len_tau09_e3[:,1],
     'new-rb-3': rb_new_len_tau09_e3[:,2],
     'new-rb-4': rb_new_len_tau09_e3[:,3],
     'new-rb-5': rb_new_len_tau09_e3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_09.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [57]:
data_new_rb_cov = {
    'new-rb-1': rb_new_cov_tau09_c3[:,0],
    'new-rb-2': rb_new_cov_tau09_c3[:,1],
    'new-rb-3': rb_new_cov_tau09_c3[:,2],
    'new-rb-4': rb_new_cov_tau09_c3[:,3],
    'new-rb-5': rb_new_cov_tau09_c3[:,4],

    
}

data_cov = pd.DataFrame(data_new_rb_cov)
data_cov.to_excel('cov_simple_cqr_09.xlsx', index=False)

#df_cov.to_excel('simu_res/res_new_rb_e_cov_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

data_new_rb_len = {
     'new-rb-1': rb_new_len_tau09_c3[:,0],
     'new-rb-2': rb_new_len_tau09_c3[:,1],
     'new-rb-3': rb_new_len_tau09_c3[:,2],
     'new-rb-4': rb_new_len_tau09_c3[:,3],
     'new-rb-5': rb_new_len_tau09_c3[:,4],

}

data_len = pd.DataFrame(data_new_rb_len)
data_len.to_excel('len_simple_cqr_09.xlsx', index=False)

#df_len.to_excel('simu_res/res_new_rb_e_len_gam0.8_tau0.5_qnum30_100to200.xlsx', index=False, engine='openpyxl')

In [27]:
n_tr = 400
gam = 0.8
T_obs = 30
seed = 2025
n_te = 310
T = 70
rep = 100
QTD_para = [20, 0.1]
alp = 0.1

res_quan_500_qnum10 = np.zeros((rep, 2))

# Parallel Calculation
def process_iteration(i,n_tr, gam, T_obs, seed, n_te, T, QTD_para, alp, tau):
 
    data_train = data_gen(N=n_tr,
                         T_obs=T_obs,
                         T=T,
                         gam=gam,
                         seed=seed+i,
                         s_init=None)
    

    data_test = data_gen_new(N=n_te,
                            T_obs=1,
                            T=T,
                            gam=gam,
                            seed=seed + i + 10000,
                            s_init=None)
    

    quan_PI_res1 = quantile_region_res(data_train=data_train,
                                      data_test=data_test, 
                                      gam=gam, 
                                      alp=alp,
                                      QTD_para=QTD_para,
                                      seed=seed+i)
    
    print(f"test num: {i}")
    print("quantile region: ")
    print(f"cov: {quan_PI_res1[0]} | length: {quan_PI_res1[1]}")
    
    return quan_PI_res1



results_qr = Parallel(n_jobs=30)(delayed(process_iteration)(i, n_tr, gam, T_obs, seed, n_te, T, QTD_para, alp, tau) for i in range(rep))

# restore data
for i in range(rep):
    res_quan_500_qnum10[i, :] = results_qr[i]


In [28]:
import pandas as pd

##### save simulation result
data_quan_cov = {
    'quantile region': res_quan_500_qnum10[:, 0]
}

df_cov = pd.DataFrame(data_quan_cov)

df_cov.to_excel('QR_cov_simple_off.xlsx', index=False, engine='openpyxl')

data_quan_len = {
    'quantile region': res_quan_500_qnum10[:, 1]
}

df_len = pd.DataFrame(data_quan_len)

df_len.to_excel('QR_len_simple_off.xlsx', index=False, engine='openpyxl')

print(df_cov.head())
print(df_len.head())

   quantile region
0         0.864516
1         0.877419
2         0.864516
3         0.864516
4         0.858065
   quantile region
0         6.990484
1         7.243581
2         6.945185
3         6.941258
4         6.875847


In [27]:
print("quantile region: ")
print("coverage probability: ", np.mean(res_quan_500_qnum10[:, 0]),
      "|  average length: ", np.mean(res_quan_500_qnum10[:, 1]))

#print([ np.mean(rb_new_cov_tau05_e[:,i]) for i in range(np.shape(rb_new_cov_tau05_e)[1])])
#print([ np.mean(rb_new_len_tau05_e[:,i]) for i in range(np.shape(rb_new_cov_tau05_e)[1])])

quantile region: 
coverage probability:  0.888 |  average length:  7.2640579569888


# Plotting simulation results

In [12]:
import pandas as pd

data_new_rb_cov = pd.read_excel('cov_simple_06.xlsx')
data_new_rb_len = pd.read_excel('len_simple_06.xlsx')
data_new_rb_cov.columns = ['k=1', 'k=2', 'k=3' , 'k=4' , 'k=5']
data_new_rb_len.columns = ['k=1', 'k=2', 'k=3' , 'k=4' , 'k=5']
data_QR_cov = pd.read_excel('QR_cov_simple_off.xlsx')
data_QR_len = pd.read_excel('QR_len_simple_off.xlsx')


#data_new_rb_cov.rename(columns={'quantile region': 'QR'}, inplace=True)
#data_new_rb_len.rename(columns={'quantile region': 'QR'}, inplace=True)
data_new_rb_cov['DRL-QR'] = data_QR_cov['quantile region']
data_new_rb_len['DRL-QR'] = data_QR_len['quantile region']

print(data_new_rb_cov.head())
print(data_new_rb_len.head())

        k=1       k=2       k=3       k=4       k=5    DRL-QR
0  0.883871  0.890323  0.900000  0.890323  0.903226  0.864516
1  0.880645  0.896774  0.896774  0.896774  0.896774  0.877419
2  0.893548  0.900000  0.900000  0.900000  0.900000  0.864516
3  0.893548  0.896774  0.896774  0.906452  0.909677  0.864516
4  0.877419  0.883871  0.903226  0.906452  0.909677  0.858065
        k=1       k=2       k=3       k=4       k=5    DRL-QR
0  7.361889  7.570965  7.603751  7.564340  7.620636  6.990484
1  7.355154  7.527859  7.614888  7.674988  7.735537  7.243581
2  7.477631  7.714179  7.784218  7.841054  7.783430  6.945185
3  7.406987  7.476894  7.554979  7.721526  7.744091  6.941258
4  7.395082  7.516494  7.672196  7.679850  7.710134  6.875847


In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

bplot_new_cov = data_new_rb_cov.boxplot(patch_artist=True,
                         medianprops={
                             'linestyle': '-',
                             'color': 'black',
                             'linewidth': 1.5
                         },
                         whiskerprops={
                             'linestyle': '--',
                             'color': 'black'
                         },
                         capprops={
                             'linestyle': '-',
                             'color': 'black'
                         },
                         boxprops={
                             'linestyle': '-',
                             'color': 'black'
                         })

colors = [
    'goldenrod', 'orange', 'gold', 'khaki', 'wheat', 'lightyellow','skyblue'
]

colors2 = [
    'darkseagreen','limegreen' ,'greenyellow','yellowgreen','lightgreen','honeydew','skyblue'
]
for patch, color in zip(bplot_new_cov.patches, colors):
    patch.set_facecolor(color)
    patch.set_linewidth(1)

bplot_new_cov.yaxis.grid(False)
bplot_new_cov.xaxis.grid(False)
bplot_new_cov.set_xlabel("Method")
bplot_new_cov.set_ylabel("Coverage Probability")

plt.axhline(y=0.90, color='red', linestyle='-', linewidth=1)
#plt.title("tau='flexible', n_tr=400, gam=0.8, quantnum=20, p_s0 estimator")
plt.ylim(0.75,1)
#plt.savefig('fig/new_rb_cov_o_gam0.8_nr100_qnum10.png')
plt.show()
plt.savefig('Ex1_off_oplicy_cp.png')

In [ ]:
plt.figure(figsize=(8, 6))

bplot_new_len = data_new_rb_len.boxplot(patch_artist=True,
                         medianprops={
                             'linestyle': '-',
                             'color': 'black',
                             'linewidth': 1.5
                         },
                         whiskerprops={
                             'linestyle': '--',
                             'color': 'black'
                         },
                         capprops={
                             'linestyle': '-',
                             'color': 'black'
                         },
                         boxprops={
                             'linestyle': '-',
                             'color': 'black'
                         })


for patch, color in zip(bplot_new_len.patches, colors):
    patch.set_facecolor(color)
    patch.set_linewidth(1)

bplot_new_len.yaxis.grid(False)
bplot_new_len.xaxis.grid(False)
bplot_new_len.set_xlabel("Method")
bplot_new_len.set_ylabel("Empirical Length")

#plt.title("tau='flexible' n_tr = 400, gam=0.8, quantnum=20, p_s0 estimator")
#plt.savefig('fig/new_len_o_gam0.8_nr100_qnum10.png')
plt.show()
plt.savefig('Ex1_off_oplicy_al.png')